# Notebook 5.4: (Optional Advanced) State Estimation for Bioreactors & NMPC with Estimated States

In our previous NMPC notebooks for the bioreactor (5.2 and 5.3), we made a crucial simplifying assumption: that all system states ($X_v, S, P, V$) were perfectly known at each control step. In reality, this is rarely true. Some states like $X_v$, $S$, and $P$ are typically measured infrequently via offline laboratory assays, and online sensors (e.g., for volume, DO, pH, capacitance) provide noisy or indirect information.

This notebook tackles this challenge by introducing **state estimation** for our nonlinear bioreactor model. We will focus on implementing an **Extended Kalman Filter (EKF)** as a practical method to estimate the unmeasured states. These estimated states will then be fed into our NMPC controller, creating an output feedback NMPC system.

**Goals of this Notebook:**
1. Discuss the challenges of state estimation in bioreactors.
2. Review the principles of the Extended Kalman Filter (EKF) for nonlinear systems.
3. Implement an EKF for the fed-batch bioreactor model, including the derivation of necessary Jacobians.
4. Simulate the bioreactor "plant" with added process and measurement noise.
5. Integrate the EKF with an NMPC controller (e.g., the glucose tracking controller from Notebook 5.2).
6. Visualize the performance: compare true states with estimated states, and analyze the closed-loop behavior of the NMPC operating on these estimates.
7. Discuss EKF tuning and the impact of estimation quality on control performance.

## 1. Importing Libraries and Re-using Code

We'll need NumPy, Matplotlib, `scipy.integrate.solve_ivp`, and CasADi (for NMPC and potentially for Jacobian calculations if not done analytically).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import casadi as ca

# Optional: for nicer plots
plt.rcParams.update({'font.size': 12, 'figure.figsize': (12, 10)}) 

# --- Bioreactor Model ODE Function and Parameters (from Notebook 5.1) ---
default_params = {
    'mu_max': 0.08, 'K_S': 0.1, 'k_d': 0.005, 'k_d_S': 0.02, 'K_d_S_coeff': 0.01,
    'Y_XS': 0.5, 'm_S': 0.01, 'alpha': 0.01, 'beta': 0.002,
    'Y_PS': 1e9, 'S_feed': 200.0
}

def bioreactor_ode_numpy(t, states, params, F_in_val): # For Scipy plant simulation
    Xv, S, P, V = states
    mu_max, K_S, k_d, k_d_S, K_d_S_coeff = params['mu_max'], params['K_S'], params['k_d'], params['k_d_S'], params['K_d_S_coeff']
    Y_XS, m_S, alpha, beta, Y_PS, S_feed = params['Y_XS'], params['m_S'], params['alpha'], params['beta'], params['Y_PS'], params['S_feed']
    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    if Y_PS > 1e6: q_S = (mu / (Y_XS + 1e-9)) + m_S
    else: q_S = (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9))
    if V < 1e-6: return [0,0,0,F_in_val]
    dilution = F_in_val / V
    dXv_dt = (mu - mu_d_val) * Xv - dilution * Xv
    dS_dt = -q_S * Xv + dilution * (S_feed - S)
    dP_dt = q_P * Xv - dilution * P
    dV_dt = F_in_val
    return [dXv_dt, dS_dt, dP_dt, dV_dt]

def bioreactor_ode_casadi(states_sx, params_sx, F_in_sx): # For CasADi NMPC
    Xv, S, P, V = states_sx[0], states_sx[1], states_sx[2], states_sx[3]
    mu_max = params_sx['mu_max']; K_S = params_sx['K_S']; k_d = params_sx['k_d']; k_d_S = params_sx['k_d_S']; K_d_S_coeff = params_sx['K_d_S_coeff']
    Y_XS = params_sx['Y_XS']; m_S = params_sx['m_S']; alpha = params_sx['alpha']; beta = params_sx['beta']; Y_PS = params_sx['Y_PS']; S_feed = params_sx['S_feed']
    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    q_S = ca.if_else(Y_PS > 1e6, (mu / (Y_XS + 1e-9)) + m_S, (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9)))
    dilution = F_in_sx / (V + 1e-9)
    dXv_dt = (mu - mu_d_val) * Xv - dilution * Xv
    dS_dt = -q_S * Xv + dilution * (S_feed - S)
    dP_dt = q_P * Xv - dilution * P
    dV_dt = F_in_sx
    return ca.vertcat(dXv_dt, dS_dt, dP_dt, dV_dt)

nx = 4; nu = 1 # Number of states and inputs

## 2. Challenges of State Estimation in Bioreactors

*   **Infrequent Offline Measurements:** Key states like Viable Cell Density ($X_v$), substrate ($S$), and product ($P$) concentrations are often measured by taking samples from the bioreactor and analyzing them in a lab. This process can take hours and might only be done once or twice a day.
*   **Noisy Online Sensors:** Online sensors (e.g., for DO, pH, temperature, volume, off-gas, capacitance) provide continuous data but are subject to noise and may only indirectly correlate with the primary states of interest.
*   **Model Uncertainty:** The biological model itself is an approximation and may have uncertain parameters or unmodeled dynamics.
*   **Nonlinearity:** The system dynamics are highly nonlinear, requiring nonlinear state estimation techniques.

An EKF attempts to address these by propagating state estimates using the nonlinear model and correcting these estimates whenever measurements (online or offline) become available.

## 3. The Extended Kalman Filter (EKF)

For a nonlinear system:
$$ x_{k+1} = f(x_k, u_k) + w_k \quad (w_k \sim \mathcal{N}(0, Q_K)) $$
$$ y_{m,k} = h(x_k, u_k) + v_k \quad (v_k \sim \mathcal{N}(0, R_K)) $$

**EKF Steps:**

**1. Time Update (Prediction):**
   $\hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1}, u_{k-1})$ (Predict state using nonlinear model)
   $P_{k|k-1} = A_k P_{k-1|k-1} A_k^T + Q_K$ (Predict error covariance using Jacobian $A_k$)
   where $A_k = \frac{\partial f}{\partial x} \Big|_{(\hat{x}_{k-1|k-1}, u_{k-1})}$

**2. Measurement Update (Correction):**
   $C_k = \frac{\partial h}{\partial x} \Big|_{(\hat{x}_{k|k-1}, u_k)}$ (Measurement Jacobian)
   $S_{inn,k} = C_k P_{k|k-1} C_k^T + R_K$ (Innovation covariance)
   $K_k = P_{k|k-1} C_k^T S_{inn,k}^{-1}$ (Compute Kalman Gain)
   $\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k (y_{m,k} - h(\hat{x}_{k|k-1}, u_k))$ (Update state estimate)
   $P_{k|k} = (I - K_k C_k) P_{k|k-1}$ (Update error covariance)

We need to define our $f(\cdot)$ (discrete-time version of our ODEs) and $h(\cdot)$ (measurement function), and their Jacobians $A_k$ and $C_k$.

In [2]:
# --- EKF Implementation ---

# For EKF, we need a discrete-time version of f(x,u)
# We can get this by integrating the ODEs over one Ts_ekf using RK4 or similar.
# Let Ts_ekf be the EKF update interval (can be same as Ts_mpc or finer).
Ts_ekf = Ts_mpc # For simplicity, use same sampling time for EKF and MPC

# Create CasADi functions for f and its Jacobian A_k = df/dx
x_s = ca.SX.sym('x_s', nx) # Symbolic state
u_s = ca.SX.sym('u_s', nu) # Symbolic input

# Define f_discrete(xk, uk) by integrating the ODE over Ts_ekf
ode_rhs_ekf = bioreactor_ode_casadi(x_s, default_params, u_s)
dae_ekf = {'x': x_s, 'p': u_s, 'ode': ode_rhs_ekf}
intg_opts_ekf = {'tf': Ts_ekf, 'simplify': True, 'number_of_finite_elements': 4}
F_discrete_integrator = ca.integrator('F_discrete', 'rk', dae_ekf, intg_opts_ekf)

x_k_plus_1_sym = F_discrete_integrator(x0=x_s, p=u_s)['xf']
f_discrete_func = ca.Function('f_discrete', [x_s, u_s], [x_k_plus_1_sym], ['xk', 'uk'], ['xk_plus_1'])

# Jacobian A_k = df/dx
Ak_jacobian_func = ca.Function('Ak_jacobian', [x_s, u_s], [ca.jacobian(x_k_plus_1_sym, x_s)], 
                               ['xk_hat', 'uk_prev'], ['Ak'])

# Measurement function h(x,u) and its Jacobian C_k = dh/dx
# Assume we measure S (glucose) and V (volume) with noise.
# Potentially also Xv (e.g. from a capacitance sensor correlated to Xv) or P (from Raman)
# For this example, let's assume we can measure S and V.
num_measured_outputs = 2 # S and V
h_measurement_sym = ca.vertcat(x_s[1], x_s[3]) # S is x_s[1], V is x_s[3]
h_measurement_func = ca.Function('h_measure', [x_s, u_s], [h_measurement_sym], 
                                 ['xk_hat_apriori', 'uk_curr'], ['yk_pred'])
Ck_jacobian_func = ca.Function('Ck_jacobian', [x_s, u_s], [ca.jacobian(h_measurement_sym, x_s)],
                               ['xk_hat_apriori', 'uk_curr'], ['Ck'])

print("EKF symbolic functions (f, Ak, h, Ck) created.")

class ExtendedKalmanFilter:
    def __init__(self, f_disc_func, h_meas_func, Ak_jac_func, Ck_jac_func, 
                 Qk_mat, Rk_mat, x_hat0, P0, n_measure_out):
        self.f = f_disc_func
        self.h = h_meas_func
        self.Ak_jac = Ak_jac_func
        self.Ck_jac = Ck_jac_func
        self.Qk = Qk_mat
        self.Rk = Rk_mat
        
        self.x_hat = np.array(x_hat0).reshape(-1,1) # a posteriori x_k|k
        self.P = np.array(P0)      # a posteriori P_k|k
        
        self.nx = self.x_hat.shape[0]
        self.ny_meas = n_measure_out

    def predict(self, uk_minus_1):
        # uk_minus_1 is numpy array
        uk_prev_num = np.array(uk_minus_1).reshape(-1,1)
        
        # Calculate Jacobian Ak at current x_hat (k-1|k-1) and u_k-1
        Ak_val = np.array(self.Ak_jac(xk_hat=self.x_hat, uk_prev=uk_prev_num)['Ak'])
        
        # Predict state using nonlinear function f
        self.x_hat_apriori = np.array(self.f(xk=self.x_hat, uk=uk_prev_num)['xk_plus_1']).reshape(-1,1)
        
        # Predict error covariance
        self.P_apriori = Ak_val @ self.P @ Ak_val.T + self.Qk # Assuming G=I for Qk here
        
    def update(self, y_measured_k, uk_current):
        # y_measured_k is numpy array (ny_meas x 1)
        # uk_current is numpy array
        uk_curr_num = np.array(uk_current).reshape(-1,1)
        y_m_num = np.array(y_measured_k).reshape(-1,1)

        # Calculate Jacobian Ck at x_hat_apriori and u_k (if h depends on u_k)
        Ck_val = np.array(self.Ck_jac(xk_hat_apriori=self.x_hat_apriori, uk_curr=uk_curr_num)['Ck'])
        
        # Predicted measurement using nonlinear function h
        y_pred_h = np.array(self.h(xk_hat_apriori=self.x_hat_apriori, uk_curr=uk_curr_num)['yk_pred']).reshape(-1,1)
        
        # Innovation and its covariance
        innovation = y_m_num - y_pred_h
        S_innovation = Ck_val @ self.P_apriori @ Ck_val.T + self.Rk
        
        # Kalman Gain
        Kk = self.P_apriori @ Ck_val.T @ np.linalg.inv(S_innovation)
        
        # Update state estimate
        self.x_hat = self.x_hat_apriori + Kk @ innovation
        
        # Update error covariance
        I_nx = np.eye(self.nx)
        self.P = (I_nx - Kk @ Ck_val) @ self.P_apriori
        self.P = 0.5 * (self.P + self.P.T) # Ensure P remains symmetric

    def step(self, uk_prev_for_predict, y_measured_k, uk_curr_for_update):
        self.predict(uk_prev_for_predict)
        self.update(y_measured_k, uk_curr_for_update)
        return self.x_hat.copy(), self.P.copy() # Return copies

NameError: name 'Ts_mpc' is not defined

### EKF Tuning Parameters ($Q_K, R_K$)

*   $R_K$: Measurement noise covariance. For our 2 measurements (S, V):
    $R_K = \begin{bmatrix} \sigma_S^2 & 0 \\ 0 & \sigma_V^2 \end{bmatrix}$
*   $Q_K$: Process noise covariance. This is a $4 \times 4$ matrix representing uncertainties in $X_v, S, P, V$ dynamics. Often chosen diagonal initially and tuned.
    $Q_K = \text{diag}(\sigma_{X_v,proc}^2, \sigma_{S,proc}^2, \sigma_{P,proc}^2, \sigma_{V,proc}^2)$

In [ ]:
# EKF Noise Parameters
sigma_S_meas = 0.1   # g/L, std dev of Glucose measurement noise
sigma_V_meas = 0.01  # L, std dev of Volume measurement noise
Rk_ekf = np.diag([sigma_S_meas**2, sigma_V_meas**2])

# Process noise sigmas (tuning parameters for EKF)
sigma_Xv_proc = 0.01 # process noise on Xv (e.g. g/L per Ts_ekf)
sigma_S_proc  = 0.05 # process noise on S
sigma_P_proc  = 0.001# process noise on P
sigma_V_proc  = 0.001# process noise on V (e.g. due to unmodelled evaporation or feed inaccuracy)
Qk_ekf = np.diag([sigma_Xv_proc**2, sigma_S_proc**2, sigma_P_proc**2, sigma_V_proc**2])

# Initial state estimate and covariance for EKF
x_hat0_ekf = np.array([0.1, 4.5, 0.01, 1.01]).reshape(nx,1) # Slightly off true initial conditions
P0_ekf = np.diag([0.05**2, 0.5**2, 0.005**2, 0.02**2])   # Uncertainty in initial guess

ekf_filter = ExtendedKalmanFilter(f_discrete_func, h_measurement_func, Ak_jacobian_func, Ck_jacobian_func,
                                  Qk_ekf, Rk_ekf, x_hat0_ekf, P0_ekf, num_measured_outputs)

## 4. NMPC with EKF: Output Feedback Control Simulation

We will use the NMPC for glucose setpoint tracking from Notebook 5.2, but now it will receive state estimates from the EKF.

In [ ]:
# --- NMPC (Glucose Tracking from 5.2) Setup ---
Ts_nmpc_ekf = Ts_ekf # NMPC runs at same rate as EKF
Np_nmpc_ekf = 12
Q_S_weight_ekf = 100.0; R_F_weight_ekf = 0.1; S_F_weight_ekf = 1.0
F_in_min_ekf = 0.0; F_in_max_ekf = 0.05; delta_F_in_max_ekf = 0.01
S_min_abs_ekf = 0.05; S_max_abs_ekf = 10.0; V_max_abs_ekf = 2.0; Xv_min_abs_ekf = 0.01
S_sp_target_ekf = 2.0

opti_nmpc_ekf = ca.Opti()
X_pred_nmpc_ekf_sym = opti_nmpc_ekf.variable(nx, Np_nmpc_ekf + 1)
U_pred_nmpc_ekf_sym = opti_nmpc_ekf.variable(nu, Np_nmpc_ekf)
x0_nmpc_ekf_param = opti_nmpc_ekf.parameter(nx)
u_prev_nmpc_ekf_param = opti_nmpc_ekf.parameter(nu)
S_sp_nmpc_ekf_param = opti_nmpc_ekf.parameter(Np_nmpc_ekf)
params_num_nmpc_ekf_ode = default_params.copy()
obj_nmpc_ekf = 0
for j in range(Np_nmpc_ekf):
    obj_nmpc_ekf += Q_S_weight_ekf * (X_pred_nmpc_ekf_sym[1, j+1] - S_sp_nmpc_ekf_param[j])**2 
    obj_nmpc_ekf += R_F_weight_ekf * (U_pred_nmpc_ekf_sym[0, j])**2 
    delta_u = U_pred_nmpc_ekf_sym[0, j] - (u_prev_nmpc_ekf_param[0] if j==0 else U_pred_nmpc_ekf_sym[0, j-1])
    obj_nmpc_ekf += S_F_weight_ekf * delta_u**2
opti_nmpc_ekf.minimize(obj_nmpc_ekf)

intg_e_nmpc = ca.integrator('intg_e_nmpc', 'rk', {'x': x_s, 'p': u_s, 'ode': bioreactor_ode_casadi(x_s, params_num_nmpc_ekf_ode, u_s)}, 
                           {'tf': Ts_nmpc_ekf, 'simplify': True, 'number_of_finite_elements': 4})
opti_nmpc_ekf.subject_to(X_pred_nmpc_ekf_sym[:,0] == x0_nmpc_ekf_param)
for j in range(Np_nmpc_ekf):
    res_n_e = intg_e_nmpc(x0=X_pred_nmpc_ekf_sym[:,j], p=U_pred_nmpc_ekf_sym[:,j])
    opti_nmpc_ekf.subject_to(X_pred_nmpc_ekf_sym[:,j+1] == res_n_e['xf'])
    opti_nmpc_ekf.subject_to(opti_nmpc_ekf.bounded(F_in_min_ekf, U_pred_nmpc_ekf_sym[0,j], F_in_max_ekf))
    delta_u_constr = U_pred_nmpc_ekf_sym[0,j] - (u_prev_nmpc_ekf_param[0] if j==0 else U_pred_nmpc_ekf_sym[0,j-1])
    opti_nmpc_ekf.subject_to(opti_nmpc_ekf.bounded(-delta_F_in_max_ekf, delta_u_constr, delta_F_in_max_ekf))
    opti_nmpc_ekf.subject_to(X_pred_nmpc_ekf_sym[0,j+1] >= Xv_min_abs_ekf)
    opti_nmpc_ekf.subject_to(opti_nmpc_ekf.bounded(S_min_abs_ekf, X_pred_nmpc_ekf_sym[1,j+1], S_max_abs_ekf))
    opti_nmpc_ekf.subject_to(X_pred_nmpc_ekf_sym[3,j+1] <= V_max_abs_ekf)

opts_setting_n_e = {'ipopt.max_iter': 100, 'ipopt.print_level': 0, 'print_time': 0, 
                    'ipopt.acceptable_tol': 1e-6, 'ipopt.acceptable_obj_change_tol': 1e-6}
opti_nmpc_ekf.solver('ipopt', opts_setting_n_e)
print("NMPC for EKF simulation formulated.")

# --- Simulation Loop with Plant, EKF, and NMPC ---
sim_time_total_ekf = 100 # hr
num_sim_steps_total_ekf = int(sim_time_total_ekf / Ts_ekf)

x_true_plant_ekf = np.array([0.1, 5.0, 0.0, 1.0]) # True initial state
u_prev_plant_ekf = np.array([0.0])

ekf_estimator = ExtendedKalmanFilter(f_discrete_func, h_measurement_func, Ak_jacobian_func, Ck_jacobian_func,
                                   Qk_ekf, Rk_ekf, x_hat0_ekf.copy(), P0_ekf.copy(), num_measured_outputs)

t_log_ekfsim = np.zeros(num_sim_steps_total_ekf + 1)
X_true_log_ekfsim = np.zeros((nx, num_sim_steps_total_ekf + 1))
X_hat_log_ekfsim = np.zeros((nx, num_sim_steps_total_ekf + 1))
Y_meas_log_ekfsim = np.zeros((num_measured_outputs, num_sim_steps_total_ekf))
U_log_ekfsim = np.zeros((nu, num_sim_steps_total_ekf))
P_diag_log_ekfsim = np.zeros((nx, num_sim_steps_total_ekf + 1))

X_true_log_ekfsim[:, 0] = x_true_plant_ekf
X_hat_log_ekfsim[:, 0] = ekf_estimator.x_hat.flatten()
P_diag_log_ekfsim[:, 0] = np.diag(ekf_estimator.P)
t_log_ekfsim[0] = 0

U_guess_nmpc_ekf = np.full((nu, Np_nmpc_ekf), 0.001)
X_guess_nmpc_ekf = np.tile(x_hat0_ekf.reshape(nx,1), (1, Np_nmpc_ekf + 1))
S_sp_horizon_ekf = np.full(Np_nmpc_ekf, S_sp_target_ekf)

print(f"Starting NMPC with EKF simulation for {num_sim_steps_total_ekf} steps...")
for k in range(num_sim_steps_total_ekf):
    current_sim_time = k * Ts_ekf
    print(f"Sim Step {k+1}/{num_sim_steps_total_ekf} (t={current_sim_time:.1f} hr)", end='\r')
    
    # 1. Plant: x_true_plant_ekf is current true state
    # 2. Measurement from plant
    y_true_S_V = np.array([x_true_plant_ekf[1], x_true_plant_ekf[3]]) # True S and V
    measurement_noise_sample = np.random.multivariate_normal([0,0], Rk_ekf)
    y_measured_ekf = y_true_S_V + measurement_noise_sample
    Y_meas_log_ekfsim[:, k] = y_measured_ekf
    
    # 3. EKF step
    # EKF predict uses u_prev_plant_ekf (input that led to x_true_plant_ekf)
    # EKF update uses y_measured_ekf and current u (u_prev_plant_ekf, as Dd=0 for measurement func h)
    x_hat_k, P_k = ekf_estimator.step(u_prev_plant_ekf, y_measured_ekf.reshape(-1,1), u_prev_plant_ekf)
    X_hat_log_ekfsim[:, k] = x_hat_k.flatten()
    P_diag_log_ekfsim[:, k] = np.diag(P_k)
    
    # 4. NMPC uses estimated state x_hat_k
    opti_nmpc_ekf.set_value(x0_nmpc_ekf_param, x_hat_k)
    opti_nmpc_ekf.set_value(u_prev_nmpc_ekf_param, u_prev_plant_ekf)
    opti_nmpc_ekf.set_value(S_sp_nmpc_ekf_param, S_sp_horizon_ekf)
    opti_nmpc_ekf.set_initial(X_pred_nmpc_ekf_sym, X_guess_nmpc_ekf)
    opti_nmpc_ekf.set_initial(U_pred_nmpc_ekf_sym, U_guess_nmpc_ekf)
    
    try:
        sol_n_e = opti_nmpc_ekf.solve()
        U_opt_n_e = sol_n_e.value(U_pred_nmpc_ekf_sym)
        X_pred_n_e = sol_n_e.value(X_pred_nmpc_ekf_sym)
        u_applied_ekf = U_opt_n_e[:, 0]
        X_guess_nmpc_ekf = np.hstack((X_pred_n_e[:, 1:], X_pred_n_e[:, -1].reshape(nx,1)))
        U_guess_nmpc_ekf = np.hstack((U_opt_n_e[:, 1:], U_opt_n_e[:, -1].reshape(nu,1)))
    except RuntimeError:
        print(f"\nNMPC Solver failed at step {k+1}. Using previous input.")
        u_applied_ekf = u_prev_plant_ekf.flatten()
        # Reset guess to be safe
        U_guess_nmpc_ekf = np.full((nu, Np_nmpc_ekf), u_prev_plant_ekf[0])
        X_guess_nmpc_ekf = np.tile(x_hat_k.reshape(nx,1), (1, Np_nmpc_ekf + 1))

    U_log_ekfsim[:, k] = u_applied_ekf
    
    # 5. Plant evolves with process noise
    process_noise_sample = np.random.multivariate_normal(np.zeros(nx), Qk_ekf).reshape(nx,1)
    # Note: Qk_ekf is for EKF. True plant process noise might be different or enter differently.
    # For simulation, assume process noise affects true states similarly to how EKF models it.
    # True ODE simulation using numpy version
    t_span_p_ekf = [current_sim_time, current_sim_time + Ts_ekf]
    plant_sol_ekf_step = solve_ivp(bioreactor_ode_numpy, t_span_p_ekf, x_true_plant_ekf, 
                                   args=(default_params, u_applied_ekf[0]), 
                                   dense_output=False, t_eval=[current_sim_time + Ts_ekf],
                                   method='LSODA')
    x_true_plant_ekf_next_no_noise = plant_sol_ekf_step.y[:,-1]
    x_true_plant_ekf = x_true_plant_ekf_next_no_noise + process_noise_sample.flatten() # Add process noise
    x_true_plant_ekf = np.maximum(x_true_plant_ekf, 0) # Enforce non-negativity
    
    X_true_log_ekfsim[:, k+1] = x_true_plant_ekf
    t_log_ekfsim[k+1] = current_sim_time + Ts_ekf
    u_prev_plant_ekf = u_applied_ekf.reshape(nu,1)
    
# Final estimate log
X_hat_log_ekfsim[:, num_sim_steps_total_ekf] = ekf_estimator.x_hat.flatten()
P_diag_log_ekfsim[:, num_sim_steps_total_ekf] = np.diag(ekf_estimator.P)

print("\nNMPC with EKF simulation finished.")

## 5. Visualizing NMPC with EKF Performance

In [ ]:
fig_ekf, axs_ekf = plt.subplots(4, 2, figsize=(16, 18), sharex='col') # Share x-axis for columns
fig_ekf.suptitle(f'Output Feedback NMPC (Glucose Tracking) with EKF', fontsize=16)
time_plot_ekf = t_log_ekfsim
time_u_plot_ekf = t_log_ekfsim[:-1]

# Column 1: States and Measurements
# Xv
axs_ekf[0,0].plot(time_plot_ekf, X_true_log_ekfsim[0, :], 'b-', label='$X_{v,true}$')
axs_ekf[0,0].plot(time_plot_ekf, X_hat_log_ekfsim[0, :], 'r--', label='$X_{v,est}$')
axs_ekf[0,0].set_ylabel('$X_v$ (g/L)'); axs_ekf[0,0].grid(True); axs_ekf[0,0].legend()
axs_ekf[0,0].set_title('Viable Cell Density')

# S (Glucose)
axs_ekf[1,0].plot(time_plot_ekf, X_true_log_ekfsim[1, :], 'b-', label='$S_{true}$')
axs_ekf[1,0].plot(time_plot_ekf, X_hat_log_ekfsim[1, :], 'r--', label='$S_{est}$')
axs_ekf[1,0].plot(time_u_plot_ekf, Y_meas_log_ekfsim[0, :], 'gx', markersize=4, alpha=0.7, label='$S_{measured}$')
axs_ekf[1,0].axhline(S_sp_target_ekf, color='k', linestyle=':', label='$S_{setpoint}$')
axs_ekf[1,0].set_ylabel('Glucose S (g/L)'); axs_ekf[1,0].grid(True); axs_ekf[1,0].legend()
axs_ekf[1,0].set_title('Glucose Concentration')

# P (Product)
axs_ekf[2,0].plot(time_plot_ekf, X_true_log_ekfsim[2, :], 'b-', label='$P_{true}$')
axs_ekf[2,0].plot(time_plot_ekf, X_hat_log_ekfsim[2, :], 'r--', label='$P_{est}$')
axs_ekf[2,0].set_ylabel('Product P (g/L)'); axs_ekf[2,0].grid(True); axs_ekf[2,0].legend()
axs_ekf[2,0].set_title('Product Concentration')

# V (Volume)
axs_ekf[3,0].plot(time_plot_ekf, X_true_log_ekfsim[3, :], 'b-', label='$V_{true}$')
axs_ekf[3,0].plot(time_plot_ekf, X_hat_log_ekfsim[3, :], 'r--', label='$V_{est}$')
axs_ekf[3,0].plot(time_u_plot_ekf, Y_meas_log_ekfsim[1, :], 'gx', markersize=4, alpha=0.7, label='$V_{measured}$')
axs_ekf[3,0].set_ylabel('Volume V (L)'); axs_ekf[3,0].set_xlabel('Time (hr)'); axs_ekf[3,0].grid(True); axs_ekf[3,0].legend()
axs_ekf[3,0].set_title('Volume')

# Column 2: Estimation Errors and Control Input
# Xv Error
axs_ekf[0,1].plot(time_plot_ekf, X_true_log_ekfsim[0, :] - X_hat_log_ekfsim[0, :], 'm-', label='Error $X_v$')
axs_ekf[0,1].plot(time_plot_ekf, 2*np.sqrt(P_diag_log_ekfsim[0,:]), 'k:', label='$\pm 2\sigma_{Xv}$')
axs_ekf[0,1].plot(time_plot_ekf, -2*np.sqrt(P_diag_log_ekfsim[0,:]), 'k:')
axs_ekf[0,1].set_ylabel('Error $X_v$'); axs_ekf[0,1].grid(True); axs_ekf[0,1].legend()
axs_ekf[0,1].set_title('Estimation Error Xv')

# S Error
axs_ekf[1,1].plot(time_plot_ekf, X_true_log_ekfsim[1, :] - X_hat_log_ekfsim[1, :], 'm-', label='Error $S$')
axs_ekf[1,1].plot(time_plot_ekf, 2*np.sqrt(P_diag_log_ekfsim[1,:]), 'k:', label='$\pm 2\sigma_S$')
axs_ekf[1,1].plot(time_plot_ekf, -2*np.sqrt(P_diag_log_ekfsim[1,:]), 'k:')
axs_ekf[1,1].set_ylabel('Error $S$'); axs_ekf[1,1].grid(True); axs_ekf[1,1].legend()
axs_ekf[1,1].set_title('Estimation Error S')

# P Error (P is not measured directly, so error might be larger)
axs_ekf[2,1].plot(time_plot_ekf, X_true_log_ekfsim[2, :] - X_hat_log_ekfsim[2, :], 'm-', label='Error $P$')
axs_ekf[2,1].plot(time_plot_ekf, 2*np.sqrt(P_diag_log_ekfsim[2,:]), 'k:', label='$\pm 2\sigma_P$')
axs_ekf[2,1].plot(time_plot_ekf, -2*np.sqrt(P_diag_log_ekfsim[2,:]), 'k:')
axs_ekf[2,1].set_ylabel('Error $P$'); axs_ekf[2,1].grid(True); axs_ekf[2,1].legend()
axs_ekf[2,1].set_title('Estimation Error P')

# Control Input F_in
axs_ekf[3,1].step(time_u_plot_ekf, U_log_ekfsim[0, :], 'k-', where='post', label='$F_{in}$ (L/hr)')
axs_ekf[3,1].axhline(F_in_max_ekf, color='m', linestyle='--', label='$F_{in,max}$')
axs_ekf[3,1].set_ylabel('Feed Rate $F_{in}$'); axs_ekf[3,1].set_xlabel('Time (hr)'); axs_ekf[3,1].grid(True); axs_ekf[3,1].legend()
axs_ekf[3,1].set_title('Control Input')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 6. Discussion and Exercises

*   **EKF Performance:** How well do the estimated states ($\hat{X_v}, \hat{S}, \hat{P}, \hat{V}$) track the true states? Are the estimation errors mostly within the $\pm 2\sigma$ bounds predicted by the EKF's covariance matrix $P$?
    *   Note that $X_v$ and $P$ are not directly measured in our setup, so their estimates might be less accurate or slower to converge than $S$ and $V$.
*   **NMPC Performance with Estimated States:** How does the glucose tracking performance of the NMPC compare to the ideal case (Notebook 5.2) where true states were known? Is it more sluggish or oscillatory due to estimation errors or delays?
*   **EKF Tuning ($Q_K, R_K$):** This is critical.
    *   Try increasing the `sigma_S_meas` and `sigma_V_meas` (elements of $R_K$). This tells the EKF the measurements are less reliable. How does this affect the estimates and control?
    *   Try decreasing the process noise sigmas (`sigma_Xv_proc`, etc., elements of $Q_K$). This tells the EKF to trust its model more. What happens if the true plant has more process noise than the EKF assumes?
    *   Try increasing the process noise sigmas. The EKF will trust measurements more. How does this impact the smoothness of estimates and the control action?
*   **Initial Guesses ($\hat{x}_0, P_0$):** How sensitive is the EKF to the initial state estimate $\hat{x}_0$ and its covariance $P_0$? Try starting the EKF with a poorer initial guess.
*   **Infrequent Measurements:** Our EKF updates at every `Ts_ekf` assuming measurements of S and V are available. How would you modify the EKF `update` step if, for example, $X_v$ and $P$ were measured only every 12 or 24 hours (offline samples)? (Hint: you might only perform the measurement update part for those specific states when their measurement arrives).
*   **Moving Horizon Estimation (MHE):** For complex bioreactors with very infrequent/delayed measurements and significant nonlinearities, MHE is often a more robust (though computationally more expensive) alternative to EKF. MHE solves an optimization problem over a moving window of past measurements to estimate states.

This notebook demonstrates that combining NMPC with a state estimator like the EKF is essential for practical application to systems where not all states are perfectly measured. The quality of control is intrinsically linked to the quality of state estimation.

## 7. Key Takeaways

*   State estimation is crucial for applying MPC to real-world bioreactors due to limited online measurements and process/measurement noise.
*   The Extended Kalman Filter (EKF) can be used to estimate states in nonlinear systems by linearizing the model at each step.
*   Implementing an EKF requires the nonlinear state transition function $f(x,u)$, the measurement function $h(x,u)$, and their Jacobians ($A_k, C_k$). CasADi can be very helpful for deriving these Jacobians.
*   Tuning the EKF's noise covariance matrices ($Q_K, R_K$) is critical for achieving good estimation performance.
*   Output feedback NMPC uses estimated states from the EKF (or other estimators) as input to the MPC optimization.
*   The performance of output feedback NMPC depends on both the NMPC design and the quality of the state estimates.

This concludes our primary bioreactor case studies using NMPC. The next optional notebook (5.5) could briefly introduce concepts for perfusion bioreactor modeling and control, offering a contrast to the fed-batch system.